In [3]:
import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import helper_func

In [4]:
full_X,x = helper_func.load_data()
users = x.shape[0]
items = x.shape[1]

In [5]:
K = 160;
rho2 = 0.1;
xi = 0.7;
rho1 = 0.01;
omega = 0.01;
omega_bar = 0.1;
tau = 10000;

NameError: global name 'np' is not defined